# Building a Spam Filter with Naive Bayes

In this project, we're going to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. Our goal is to write a program that classifies new messages with an accuracy greater than 80% — so we expect that more than 80% of the new messages will be classified correctly as spam or ham (non-spam).

To train the algorithm, we'll use a dataset of 5,572 SMS messages that are already classified by humans. The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). The data collection process is described in more details on this page, where you can also find some of the papers authored by Tiago A. Almeida and José María Gómez Hidalgo.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("D:/Library/datasci/datasets/SMSSpamCollection.csv", sep = '\t', header = None,
                names = ['Label', 'SMS'])

In [3]:
df.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Label   5572 non-null   object
 1   SMS     5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [5]:
df.Label.value_counts()/5572

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

4825 of our 5572 entries are labeled as not spam. We can do a little bit of calculation to get that we have approximately 86.6% non-spam, and 13.4% spam.


In [6]:
#Import a splitter from sklearn
from sklearn.model_selection import train_test_split

In [7]:
X,y = df.iloc[:,1:], df.iloc[:,0]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                   test_size = 0.2,
                                                   random_state = 1,
                                                   stratify = y)

In [9]:
y_train.value_counts()/y_train.shape[0]

ham     0.865829
spam    0.134171
Name: Label, dtype: float64

In [10]:
y_test.value_counts()/y_test.shape[0]

ham     0.866368
spam    0.133632
Name: Label, dtype: float64

We see that the training and test sets have roughly equal proportions of spam and non-spam

# Cleaning Data

We'll remove all of the punctuation and make all of the messages lower case in an attempt to normalize the words.

In [11]:
#The regex \W will grab all non words. 
X_train.SMS = X_train.SMS.str.replace(r'\W', ' ', regex = True).str.lower()


In [12]:
X_train.SMS.head()

2357                          no  he joined today itself 
5568                 will ü b going to esplanade fr home 
2987    reply to win  100 weekly  what professional sp...
951            awesome  lemme know whenever you re around
648     private  your 2003 account statement for shows...
Name: SMS, dtype: object

In [13]:
#same as above
X_test.SMS = X_test.SMS.str.replace(r'\W', ' ', regex = True).str.lower()

# Create a Vocabulary list.
We will combine the training and test set to create a more robust vocabulary. Since we have a way to treat words that we have never seen before in the Naive Bayes algorithm, I see no reason not to use all of the data available to simply capture the words.

In [14]:
#combine to make a dictionary. To be honest though this may be cheating.
#it may make more sense to make the dictionary purely off of the test set.
xwhole = pd.concat([X_test,X_train])

In [15]:
xwhole.SMS = xwhole.SMS.str.split()

In [16]:
vocabulary = []
for string in xwhole.SMS:
    for word in string:
        vocabulary.append(word)
            
vocabulary = list(set(vocabulary))

In [17]:
#Check for repeates to make sure it worked.
len(vocabulary)

8753

# Create the dictionary and transform into a dataset
We are now going to create a dictionary using our vocabulary set. We will essentially make it a series of zeroes equal to the length of our entire dataset. What we will then do is iterate through the messages and increase the count at a given index in our dictionary by the number of times the word appears in the sms message.

In [274]:
word_counts_per_sms = {unique_word: [0] * len(xwhole['SMS']) for unique_word in vocabulary}

In [276]:
for index, sms in zip(xwhole.index,xwhole.SMS):
    for word in sms:
        word_counts_per_sms[word][index] += 1


In [217]:
#first5pairs = {k: word_counts_per_sms[k] for k in sorted(word_counts_per_sms.keys())[:5]}

In [281]:
word_counts = pd.DataFrame(word_counts_per_sms,)
word_counts.head()

,8,shes,agalla,knock,mobile,at,laready,mymoby,0a,zealand,...,extreme,neighbor,pack,elliot,9061100010,dream,valuable,smsco,pobox334,ec2a
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [309]:
X_train_clean.loc[2357][X_train_clean.loc[2357] == 1]

he        1
itself    1
no        1
today     1
joined    1
Name: 2357, dtype: object

In [286]:
X_train_clean = pd.concat([xwhole, word_counts], axis = 1)
X_train_clean.head()

,SMS,8,shes,agalla,knock,mobile,at,laready,mymoby,0a,...,extreme,neighbor,pack,elliot,9061100010,dream,valuable,smsco,pobox334,ec2a
977,"[ok, i, shall, talk, to, him]",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4688,"[eatin, my, lunch]",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
142,"[sir, waiting, for, your, mail]",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4708,"[wif, my, family, booking, tour, package]",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5335,"[no, it, s, not, pride, i, m, almost, lt, gt, ...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
# X_train['id'] = X_train.index
# X_test['id'] = X_test.index

In [311]:
X_train_final = pd.concat([X_train, X_train_clean], join = "inner", axis = 1)
X_train_final = X_train_final.iloc[:,1:]

In [312]:
X_train_final.loc[2357][X_train_final.loc[2357] == 1]

he        1
itself    1
no        1
today     1
joined    1
Name: 2357, dtype: object

In [313]:
X_train_final.head()

,SMS,8,shes,agalla,knock,mobile,at,laready,mymoby,0a,...,extreme,neighbor,pack,elliot,9061100010,dream,valuable,smsco,pobox334,ec2a
2357,"[no, he, joined, today, itself]",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5568,"[will, ü, b, going, to, esplanade, fr, home]",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2987,"[reply, to, win, 100, weekly, what, profession...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
951,"[awesome, lemme, know, whenever, you, re, around]",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
648,"[private, your, 2003, account, statement, for,...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Create the algorithm
We now have a good training and test set, it is time to actually make the algorithm for classification. To get started we will be working towards calculating the probability that a message is spam given the contents of the message. We also want to calculate the probability that a message is not spam given the contents of the message.

As we are looking to calculate:
\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} \\
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

To do so we will need to calculate:
* $P(Spam)$ The probability of spam. We have this from above, 0.134
* $P(Ham)$ The probability of ham. We also have this from above. 0.866
* $N_{spam}$ The count of total words in all spam messages
* $N_{ham}$ The count of total words in all non-spam messages
* $N_{vocabulary}$ The count of total words in the vocabulary. We already have this it is 8753.

So we now will do some counting.

In [314]:
#Create spam and ham training sets.
#Spam and ham training sets.
spam_id = y_train[y_train == "spam"].index
ham_id = y_train[y_train == "ham"].index

X_train_spam = X_train_final.loc[spam_id]
X_train_ham = X_train_final.loc[ham_id]

#Nspam
N_spam = X_train_spam.SMS.apply(len).sum()
N_ham = X_train_ham.SMS.apply(len).sum()

#Nvocabulary
N_vocabulary = 8753

#Alpha
a = 1

#probs
pspam = len(spam_id)/len(X_train_final)
pham = len(ham_id)/len(X_train_final)

# Calculating Parameters

We need to calculate $P(w_i|Spam)$ and $P(w_i|Ham)$. These two conditional probabilities form the basis of our comparison for the Naive Bayes algorithm. Again each of these probabilities is just the probability that a certain word exists in a message given that it is spam or non-spam. These can be calculated with the following:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} \\
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

To do so we will create a spam dictionary and a non-spam dictionary, then fill in the probabilities using our constants calculated above.

In [315]:
# Initialize words and set probabilities to 0
spam_dict = {}
ham_dict = {}

for word in vocabulary:
    spam_dict[word] = 0
    ham_dict[word] = 0 

In [316]:
#Calculate probabilities for SPAM and for HAM

for word in vocabulary:
    prob_spam = (X_train_spam[word].sum() + a) / (N_spam + a * N_vocabulary)
    prob_ham = (X_train_ham[word].sum() + a) / (N_ham + a * N_vocabulary)
    spam_dict[word] = prob_spam
    ham_dict[word] = prob_ham

# Classifying A New Message
Now that we have all our parameters calculated, we can start creating the spam filter. The spam filter can be understood as a function that:

* Takes in as input a new message $(w_1, w_2, ..., w_n)$.
* Calculates $P(Spam|w_1, w_2, ..., w_n)$ and $P(Ham|w_1, w_2, ..., w_n)$.
* Compares the values of $P(Spam|w_1, w_2, ..., w_n)$ and $P(Ham|w_1, w_2, ..., w_n)$, and:
    * If $P(Ham|w_1, w_2, ..., w_n) > P(Spam|w_1, w_2, ..., w_n)$, then the message is classified as ham.
    * If $P(Ham|w_1, w_2, ..., w_n) < P(Spam|w_1, w_2, ..., w_n)$, then the message is classified as spam.
    * If $P(Ham|w_1, w_2, ..., w_n) = P(Spam|w_1, w_2, ..., w_n)$, then the algorithm may request human help.

In [317]:
##Classifier function.

import re

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()
    
    p_spam_given_message = pspam
    p_ham_given_message = pham
    
    for word in message:
        if word in spam_dict:
            p_spam_given_message *= spam_dict[word]
            
        if word in ham_dict:
            p_ham_given_message *= ham_dict[word]
    

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

Some quick testing:

In [318]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 6.825887390849328e-26
P(Ham|message): 1.909125784210236e-27
Label: Spam


In [326]:
classify('Sounds good, Tom, then see u there')

P(Spam|message): 3.983681276035583e-25
P(Ham|message): 2.604255852554336e-21
Label: Ham


# Measuring the Spam Filter's Accuracy

The two results above look promising, but let's see how well the filter does on our test set, which has 1,115 messages.

We'll start by writing a function that returns classification labels instead of printing them.

In [320]:
# Slightly modify function to return the label for test validation
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = pspam
    p_ham_given_message = pham

    for word in message:
        if word in spam_dict:
            p_spam_given_message *= spam_dict[word]

        if word in ham_dict:
            p_ham_given_message *= ham_dict[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [321]:
X_test['predicted'] = X_test['SMS'].apply(classify_test_set)

In [322]:
final = pd.concat([X_test,y_test], join = 'inner', axis = 1)

In [323]:
correct = (final.predicted == final.Label).sum()
total = len(final)

accuracy = correct/total
print("Correct:", correct)
print("total:", total)
print("Accuracy:",accuracy)

Correct: 1091
total: 1115
Accuracy: 0.97847533632287


Our accuracy on the filter is roughly 98% which is quite good and shows a large increase over the baseline of 86% accuracy of simply labeling everything non-spam.

# Some Potential Next Steps

In this project, we managed to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. The filter had an accuracy of 98.74% on the test set we used, which is a pretty good result. Our initial goal was an accuracy of over 86%, and we managed to do way better than that.

Next steps could include:

* Analyze the  messages that were classified incorrectly and try to figure out why the algorithm classified them incorrectly
* Make the filtering process more complex by making the algorithm sensitive to letter case